In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# default_exp eda

# 事件抽取
http://challenge.xfyun.cn/topic/info?type=hotspot

https://zhuanlan.zhihu.com/p/150190165

## 数据说明
1. 初赛数据说明：

本次比赛初赛为参赛选手提供了6958条中文句子，及其9644条提取结果（存在一对多的情况）：

1.1训练集：共5758条句子，包含句子中对应的触发词、论元等，用于竞赛模型训练。

1.2测试集：共1200条句子。

2. 复赛数据说明：

本次比赛复赛为参赛选手提供了3335条中文句子，及其3384条提取结果（存在一对多的情况）：

2.1训练集：共2456条句子，包含句子中对应的触发词、论元及其角色、事件属性等，用于竞赛模型训练。

2.2测试集：共879条句子。
## 评价指标

本模型依据提交的结果文件，采用F值进行评价。

### 事件触发词及论元抽取评价指标 
对于事件触发词及论元抽取，使用触发词进行事件对齐，对于触发词匹配的事件，计算论元F值。最终F值包括论元与触发词（将触发词当做一种论元计算得分）。论元F值为严格F值与松弛F值的平均得分。

3.1.1 严格F值：预测论元与标注论元必须完全匹配（类型必须正确）

论元准确率Pspan = 预测论元和标注论元匹配的个数 / 预测论元个数

论元召回率Rspan = 预测论元和标注论元匹配的个数 / 标注论元个数

论元F值 F1span = 2 Pspan Rspan /( Pspan + Rspan)

3.1.2 松弛F值：预测论元与标注论元存在字符级别匹配也能得到部分分数（类型必须正确）

论元准确率Pchar = 预测论元和标注论元匹配的字符数 / 预测论元字符数

论元召回率Rchar = 预测论元和标注论元匹配的字符数 / 标注论元字符数

论元F值 F1char = 2 Pchar Rchar /( Pchar + Rchar)

3.1.3 最终得分：F1 = (F1span + F1char) / 2

### 事件属性抽取评价指标

对于事件属性抽取，使用F值进行评价。

属性准确率P = 预测属性和标注属性匹配的个数/ 预测属性个数

属性召回率R = 预测属性和标注属性匹配的个数 / 标注属性个数

属性F值 F1= 2 P R /( P+ R)

# 解题思路
## mine
因为给的数据量很少，只能用预训练模型，finetune。当然预训练模型也不用很复杂。

很好的图像练手的比赛！
## 大佬


# lib导入

In [1]:
# export
import os
import glob
from code.config import * 
from loguru import logger
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

from sklearn.metrics import classification_report, accuracy_score
import sys
sys.path.append('..')
from mylib.utils.pickle import PickleWrapper

In [2]:

import seaborn as sns
sns.set(font='Arial Unicode MS')  # 解决Seaborn中文显示问题
from mylib.data.processing.eda import describe

In [3]:
args.DATA_DIR

'../../data/contest/20kdxf_event/'

In [4]:
!mkdir {args.DATA_DIR}

In [6]:
!open {args.DATA_DIR}

In [7]:
!ls {args.DATA_DIR}

test
train
事件抽取挑战赛_事件抽取挑战赛提交样例.csv


# EDA

In [4]:
train_raw = pd.read_csv(args.DATA_DIR+'train/train.csv')

In [5]:
train_raw.sample(20)

,id,news,trigger,object,subject,time,location
1270,1130,为此，该市商务部门联合市直机关工委、市委组织部积极组织引导社区与各超市、菜市场、农产品批发市...,组织,市委组织部,社区与各超市、菜市场、农产品批发市场对接,NaN,NaN
5498,5698,通过在北京大学开设的课程以及在北京高等师范学校的兼课和在武汉、上海、成都等地高校的讲学活动，...,培养,李大钊,第一代熟悉马克思主义唯物史观的高等专门人才,NaN,中国
1608,1375,她告诉记者，自己的成长不仅是靠自身努力而得来，更重要的是，得益于中国铁路日渐完善的服务体系,得益,自己的成长,中国铁路日渐完善的服务体系,NaN,NaN
3757,3874,原标题：4月8日起，武汉恢复巡游出租车运营今日，记者从武汉市防疫指挥部获悉，为做好解除离汉通...,恢复,武汉,巡游出租车运营,4月8日起,NaN
3715,3846,原标题：军方援鄂专家组组长：我们发现个别健康人携带病毒今晚(6日)的《新闻1+1》，白岩松连...,连线,白岩松,军方援鄂前方专家组组长、解放军总医院呼吸科专家刘又宁,6日,NaN
7173,7058,印度总理莫迪上3月25日宣布印度全国封城三周，严禁印度13亿人口离开家门外出活动，2日是印度...,宣布,印度总理莫迪,印度全国封城三周,上3月25日,NaN
4986,5256,在这种逆境中，戴笠养成了揣摩蒋介石心理的习惯，他注意分析政治军事形势和蒋介石的情报需求，不辞...,送上,戴笠,他的工作成果,NaN,NaN
3554,3649,他接受法新社记者电话采访时说，周围的人先前基本没有遵守印度政府的居家要求，现在塔拉维出现病例...,接受,他,法新社记者电话采访,NaN,NaN
3648,3764,3月24日湖北省新冠肺炎疫情防控指挥部24日发布通告说，经中央批准，武汉市将于4月8日起解除...,发布,湖北省新冠肺炎疫情防控指挥部,通告,3月24日,NaN
7194,7081,1500余名美国胸科医师学会会员注册参加了研讨会,参加,1500余名美国胸科医师学会会员,研讨会,NaN,NaN


In [5]:
test_raw = pd.read_csv(args.DATA_DIR+'test/test.csv')

In [13]:
test_raw.head()

,id,news
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设


In [14]:
test_raw.shape

(1200, 2)

## 脏数据

### time 脏数据

邓小平 交通运输部 农业农村部 颐年堂之后 资委会领导人孙越崎

In [6]:
tmp = train_raw.time[train_raw.time.notnull()].copy()

In [7]:
tmp[~tmp.str.contains(r'[1-9]|[一二三十]')]

74               今年
222              凌晨
228              凌晨
526              今年
601           １８日下午
620            春节期间
822            本世纪初
842             去年起
879            除夕深夜
880            除夕深夜
1160             春节
1276          农业农村部
1302             今年
1303             今年
1304             今年
1305             今年
1408            今年初
1416            近两年
1430             昨天
1441           这个春节
1488             春秋
1663             除夕
1678             除夕
1778             今年
2023             去年
2041          今年上半年
2044            今年初
2077          交通运输部
2130           今年年初
2132             今年
2393            除夕夜
2804            周日晚
2812           今天下午
3034           今天下午
3389             明末
3390             明末
3540             今年
3789        清明节假日期间
3916             次日
3945           两个多月
4023           今天下午
4071             次日
4073             次日
4074             次日
4075             次日
4076             次日
4077             次日
4325          １９３０年
4326          １９３０年
4327          １９３０年


In [6]:
train_raw.loc[train_raw.time=='邓小平', 'object'] = '邓小平'

train_raw.loc[train_raw.time=='邓小平', 'time'] = None

train_raw.loc[train_raw.time=='交通运输部', 'object'] = '交通运输部'

train_raw.loc[train_raw.time=='交通运输部', 'time'] = None

train_raw.loc[train_raw.time=='农业农村部', 'object'] = '农业农村部'

train_raw.loc[train_raw.time=='农业农村部', 'time'] = None

train_raw.loc[train_raw.time=='资委会领导人孙越崎', 'object'] = '资委会领导人孙越崎'

train_raw.loc[train_raw.time=='资委会领导人孙越崎', 'time'] = '6月'

train_raw.loc[train_raw.time=='颐年堂之后', 'time'] = None

### 还有一些奇怪的编码方式 需要确认一下模型的识别效果

１９６７年２月１５日

经验证，或影响transformer的tokenize结果！

In [7]:
strange_dict = {
    '１': '1',
    '９': '9',
    '６': '6',
    '７': '7',
    '２': '2',
    '５': '5',
    '３': '3',
    '４': '4',
    '８': '8',
    '０': '0',
    '\u200d': ' ',
    '■': ' ',
    '▼': ' ',
    '△': ' ',
#     '': ' ',
}

#### 验证test中是否有这些奇怪的编码
没有 nice!!!!!

In [8]:
''.join(strange_dict.keys())

'１９６７２５３４８０\u200d■▼△'

In [9]:
test_raw[test_raw.news.str.contains(fr'[{"".join(strange_dict.keys())}]')]

,id,news
169,8276,△案件再审相关文书“我举报2010年孙大强交通肇事的案件，真凶并非孙大强，而是他的亲弟弟孙大...
525,8974,■社论“新基建”不仅为疫情后经济复苏提供助力，更可以从根本上加速我国经济换挡升级，提升中国经...
770,9347,■来论3月7日晚间，福建泉州南环路附近一处酒店发生倒塌


In [10]:
r = str.maketrans(strange_dict)
r

{65297: '1',
 65305: '9',
 65302: '6',
 65303: '7',
 65298: '2',
 65301: '5',
 65299: '3',
 65300: '4',
 65304: '8',
 65296: '0',
 8205: ' ',
 9632: ' ',
 9660: ' ',
 9651: ' '}

In [11]:
train_raw['time'].loc[4331]

'１９３２年'

In [11]:
train_raw['news'] = train_raw['news'].map(lambda s: s.translate(r))

train_raw['time'] = train_raw['time'].map(lambda s: s.translate(r), na_action='ignore')

train_raw['trigger'] = train_raw['trigger'].map(lambda s: s.translate(r), na_action='ignore')
train_raw['object'] = train_raw['object'].map(lambda s: s.translate(r), na_action='ignore')
train_raw['subject'] = train_raw['subject'].map(lambda s: s.translate(r), na_action='ignore')
train_raw['location'] = train_raw['location'].map(lambda s: s.translate(r), na_action='ignore')

In [12]:
test_raw['news'] = test_raw['news'].map(lambda s: s.translate(r))

In [22]:
train_raw['time'].loc[4331]

'1932年'

In [13]:
train_raw.news[train_raw.news.str.contains('\u200d')]

Series([], Name: news, dtype: object)

In [14]:
test_raw.news[test_raw.news.str.contains('\u200d')]

Series([], Name: news, dtype: object)

In [15]:
train_raw.news[train_raw.news.str.contains('■')]

Series([], Name: news, dtype: object)

In [16]:
test_raw.news[test_raw.news.str.contains('■')]

Series([], Name: news, dtype: object)

In [17]:
train_raw.news[train_raw.news.str.contains('▼')]

Series([], Name: news, dtype: object)

In [18]:
test_raw.news[test_raw.news.str.contains('▼')]

Series([], Name: news, dtype: object)

In [19]:
train_raw.news[train_raw.news.str.contains('△')]

Series([], Name: news, dtype: object)

In [20]:
test_raw.news[test_raw.news.str.contains('△')]

Series([], Name: news, dtype: object)

### 修改其它一些错误

In [21]:
train_raw.loc[0]

id                                                          1
news        新华社北京4月7日电（记者王优玲）人力资源社会保障部7日启动第七届大中城市联合招聘高校毕业生...
trigger                                                    启动
object                                              人力资源社会保障部
subject                                第七届大中城市联合招聘高校毕业生春季专场活动
time                                                       7日
location                                                  NaN
Name: 0, dtype: object

In [22]:
train_raw.loc[0, 'time'] = '4月7日'

In [23]:
train_raw.loc[45]

id                                                         35
news        新华社北京4月7日电（记者孙少龙）记者7日从民政部获悉，民政部近日对各地提出指导意见，要求各...
trigger                                                    提出
object                                                    民政部
subject                                                  指导意见
time                                                       7日
location                                                  NaN
Name: 45, dtype: object

In [24]:
train_raw.loc[45, 'time'] = '4月7日'

In [25]:
train_raw.loc[6247]

id                               6307
news        后来，他在宁都会议上曾支持让毛泽东留在前方指挥红军
trigger                            支持
object                              他
subject                  让毛泽东留在前方指挥红军
time                              NaN
location                          宁都上
Name: 6247, dtype: object

In [26]:
train_raw.loc[6247, 'location'] = '宁都会议上'

In [27]:
train_raw.loc[5670]

id                                  5841
news        1964年6～7月，中国连续成功地发射了3枚中程运载火箭
trigger                               发射
object                                中国
subject                         3枚中程运载火箭
time                           1964年6~7月
location                             NaN
Name: 5670, dtype: object

In [28]:
train_raw.loc[5670, 'time'] = '1964年6～7月'

In [29]:
train_raw.loc[5477]

id                                                       5685
news        为了避免在阿拉善旗边卡被发现，奥希洛夫等人改变行走路线，绕道北银根、查汉扎德盖、图克木、罕乌...
trigger                                                    改变
object                                                 奥希洛夫等人
subject                                                  行走路线
time                                                      NaN
location                                                 阿拉善边
Name: 5477, dtype: object

In [30]:
train_raw.loc[5477, 'location'] = '阿拉善旗边卡'

In [31]:
train_raw.loc[5308]

id                                                       5534
news        一1959年7月中旬，在党中央召开的庐山会议上，彭德怀根据他深入群众、实地视察的所见所闻，对...
trigger                                                    提出
object                                                    彭德怀
subject                                              自己的看法和意见
time                                               一1959年7月中旬
location                                                  庐山上
Name: 5308, dtype: object

In [32]:
train_raw.loc[5308, 'location'] = '庐山会议上'

In [33]:
train_raw.loc[5280]

id                                          5510
news        与许多革命元勋一样，贺龙在成为共产主义者之前，接受了孙中山三民主义的熏陶
trigger                                       接受
object                                        贺龙
subject                               孙中山三民主义的熏陶
time                                         NaN
location                                    共产之前
Name: 5280, dtype: object

In [34]:
train_raw.loc[5280, 'location'] = None

In [35]:
train_raw.loc[4062]

id                                              4248
news        这段视频公开了不少中央指导组在湖北战疫中的珍贵工作画面，也披露了孙春兰的工作情况
trigger                                           披露
object                                          这段视频
subject                                     孙春兰的工作情况
time                                             NaN
location                                         湖北中
Name: 4062, dtype: object

In [36]:
train_raw.loc[4062, 'location'] = '湖北战疫中'

In [37]:
train_raw.loc[3792]

id                                                       3919
news        厦门市宣布，2020年6月30日前，国有A级景区向市民游客免费开放，对厦门市旅行社组织招徕福...
trigger                                                  免费开放
object                                                 国有A级景区
subject                                                   NaN
time                                              2020年6月30日前
location                                                  福建内
Name: 3792, dtype: object

In [38]:
train_raw.loc[3792, 'location'] = '福建省内'

In [39]:
train_raw.loc[1512]

id                                                       1294
news        2017年7－8月，华贵人寿在银保业务经营过程中，制定并执行营销方案，向投保人赠送价值合计1...
trigger                                                    制定
object                                                   华贵人寿
subject                                                  营销方案
time                                                2017年7-8月
location                                                  NaN
Name: 1512, dtype: object

In [40]:
train_raw.loc[1512, 'time'] = '2017年7－8月'

In [41]:
train_raw.loc[6852]

id                                                       6777
news        港珠澳大桥香港口岸明起缩短清关服务时间（中新社资料图片）大公文汇全媒体报道：香港特区政府发言...
trigger                                                    公布
object                                              香港特区政府发言人
subject              港珠澳大桥香港口岸的旅客清关服务时间将于明日(5日)凌晨零时起调整，直至另行通告
time                                                    今日(4日
location                                                  NaN
Name: 6852, dtype: object

In [42]:
train_raw.loc[6908]

id                                                       6826
news        星岛环球网消息：大公文汇全媒体报道，香港财政司司长陈茂波今日(5日)发表题为《撑企业保就业》的网志
trigger                                                    发表
object                                             香港财政司司长陈茂波
subject                                         题为《撑企业保就业》的网志
time                                                    今日(5日
location                                                星岛环球网
Name: 6908, dtype: object

In [43]:
train_raw.time[train_raw.news.str.contains('\(5日\)')]

6852    今日(4日
6908    今日(5日
Name: time, dtype: object

In [79]:
train_raw.sample(20)

,id,news,trigger,object,subject,time,location
1615,1379,辽宁方面也明确开展联盟地区（辽宁省）药品集中采购品种未中选药品梯度降价工作,开展,辽宁方面,联盟地区（辽宁省）药品集中采购品种未中选药品梯度降价工作,NaN,NaN
4392,4667,1972年8月3日，邓小平再次致信毛泽东，一方面继续批判林彪罪行，同时进一步检查自己的错误，...,提出,邓小平,愿为党和人民做一点工作的要求,1972年8月3日,NaN
2226,2125,小米公司在业内率先提出“全面屏”概念，其发售的MIX系列手机随后被各大手机厂商纷纷效仿，MI...,提出,小米公司,“全面屏”概念,NaN,NaN
2445,2404,“有人说我们村造房子用的混凝土是劣质的，还说是我们村书记许时新介绍的，不知道他从中得到什么好...,移交至,许时新,富阳区纪委监委,NaN,NaN
4441,4723,6月18日，北大的聂元梓写大字报反对八条指示，并揪斗了校领导陆平、彭云和著名历史学教授翦伯赞等人,反对,北大的聂元梓,八条指示,6月18日,NaN
4181,4410,原标题：国家卫健委为16类重点人群打造“个性化”防控方案为有效防止聚集性疫情的发生，将疫情风...,打造,国家卫健委,“个性化”防控方案,NaN,NaN
288,265,◆国务院联防联控机制召开新闻发布会，介绍新型冠状病毒感染的肺炎疫情防控工作中孕产妇、婴幼儿和...,召开,国务院联防联控机制,新闻发布会,NaN,NaN
1059,999,最高检14日消息，河北省人民政府原党组成员、副省长李谦涉嫌受贿一案，由国家监察委员会调查终结...,调查,河北省人民政府原党组成员、副省长李谦,国家监察委员会,NaN,NaN
5837,5988,与此同时，刘少奇抓住由于日军侵占造成敌后政权真空的时机，委派干部出任专员、县长，并力主建立共...,出任,委派干部,专员、县长,NaN,NaN
6645,6621,林郑月娥说，食物环境卫生署、警务处、康乐及文化事务署、民政事务总署等发牌机构、执法部门过去两...,作出,食物环境卫生署、警务处、康乐及文化事务署、民政事务总署等发牌机构、执法部门,400多次口头劝喻,NaN,NaN


In [44]:
train_raw.to_pickle('train_raw.pkl')

## labeled

In [45]:
train_raw = pd.read_pickle('train_raw.pkl')

In [46]:
train_raw = train_raw.fillna('')

In [47]:
marks_list = []
for line in train_raw.itertuples():
    marks = ['O'] * len(line.news)
    for e, obj in enumerate('trigger object subject time location'.split()):
#         print(line[e+3])
        if line[e+3] == '': continue 
        start = line.news.find(line[e+3])
        if start != -1:
            marks[start] = f'B-{obj}'
            for i in range(len(line[e+3])-1):
                marks[start+i+1] = f'I-{obj}'
    marks_list.append(marks)
#     print(marks)
#     break

In [48]:
len(marks_list)

8000

In [49]:
news_list = train_raw.news.map(list).tolist()

In [50]:
results = []
for news, marks in zip(news_list, marks_list):
    ll = []
    for n, m in zip(news, marks):
        ll.append(f'{n} {m}')
    results.append('\n'.join(ll))

In [51]:
len(results)

8000

In [67]:
with open('kdxf_event.txt', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(results))

In [52]:
with open('train.txt', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(results[:7000]))

In [53]:
with open('dev.txt', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(results[7000:]))

In [54]:
news_list[0]

['新',
 '华',
 '社',
 '北',
 '京',
 '4',
 '月',
 '7',
 '日',
 '电',
 '（',
 '记',
 '者',
 '王',
 '优',
 '玲',
 '）',
 '人',
 '力',
 '资',
 '源',
 '社',
 '会',
 '保',
 '障',
 '部',
 '7',
 '日',
 '启',
 '动',
 '第',
 '七',
 '届',
 '大',
 '中',
 '城',
 '市',
 '联',
 '合',
 '招',
 '聘',
 '高',
 '校',
 '毕',
 '业',
 '生',
 '春',
 '季',
 '专',
 '场',
 '活',
 '动']

In [55]:
marks_list[2]

['B-object',
 'I-object',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-trigger',
 'I-trigger',
 'O',
 'O',
 'B-subject',
 'I-subject',
 'I-subject',
 'I-subject',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [65]:
sub_raw = pd.read_csv(args.DATA_DIR+'事件抽取挑战赛_事件抽取挑战赛提交样例.csv')

In [66]:
sub_raw.shape

(1644, 6)

In [33]:
sub_raw.head()

,id,trigger,object,subject,time,location
0,7953,恢复,深航,湖北其他城市的航班运力,NaN,NaN
1,7955,恢复,深航,湖北其他城市的航班运力,NaN,NaN
2,7956,恢复,深航,湖北其他城市的航班运力,NaN,NaN
3,7956,恢复,深航,湖北其他城市的航班运力,NaN,NaN
4,7959,恢复,深航,湖北其他城市的航班运力,NaN,NaN


In [56]:
test = []
for i in test_raw.news.map(list).tolist():
    test.append('\n'.join(i))

In [57]:
with open('test.txt', 'w', encoding='utf8') as f:
    f.write('\n\n'.join(test))

## gen预测结果

In [61]:
# fp = 'test_predictions_4_342.txt'
with open('test_predictions_modify0818.txt', 'r', encoding='utf8') as f:
    lines = f.readlines()

marks = [line.strip().split()[-1] if line.strip() else '##' for line in lines]



marks = ' '.join(marks)

marks = marks.split(' ## ')

print(len(marks))

preds = []
for ii, marks_ in enumerate(marks):
#     print(marks_)
#     print(test_raw.iloc[ii]['news'])
    d = {
        'trigger': None,
        'object': None,
        'time': None,
        'location': None,
    }
    start = -1
    range_ = 1
    label = None
    for i, mark in enumerate(marks_.split()):
        d['id'] = test_raw.iloc[ii]['id']
        if mark.startswith('B'):
            if label and start != -1:
                d[label] = test_raw.iloc[ii]['news'][start: start+range_]
            start = i
            range_=1
            label = mark.split('-')[-1]
        elif mark.startswith('I'):
            if label == mark.split('-')[-1]:
                range_ += 1
            elif label:
                d[label] = test_raw.iloc[ii]['news'][start: start+range_]
                start = -1
                range_ = 1
                label = None
        elif mark.startswith('O'):
            if label:
                d[label] = test_raw.iloc[ii]['news'][start: start+range_]
                start = -1
                range_ = 1
                label = None
    preds.append(d)
#     print(d)
#         break
#     break

sub_ = pd.DataFrame(preds)

test_raw.head()

sub_[sub_raw.columns].to_csv(args.DATA_DIR+'sub__modify0818.csv', index=False)

169


In [63]:
sub_.shape

(169, 6)

In [64]:
sub_.sample(20)

,trigger,object,time,location,id,subject
40,发布,广州恒大淘宝足球俱乐部,1,None,8022,公告
67,None,意甲联盟和意大利足协,None,None,8072,降薪一事
56,捐赠,西班牙人俱乐部,今年2月份,None,8055,45万个口罩
109,发布,中国球员武磊,北京时间1日晚,None,8135,NaN
126,到达,小罗,None,None,8174,NaN
84,None,德国足协,None,None,8101,NaN
7,发布,None,None,None,7965,志愿者招募信息
27,公布,广东高院,None,None,8005,NaN
105,发表,欧足联官网,4月1日,None,8130,声
29,进行,巴黎,3月10日,None,8008,欧冠赛前的最后一场训练


In [13]:
train_raw.news.str.len().describe()

count    8000.000000
mean       71.780875
std        39.728918
min        10.000000
25%        45.000000
50%        63.000000
75%        88.000000
max       397.000000
Name: news, dtype: float64

In [14]:
test_raw.news.str.len().describe()

count    1200.000000
mean       62.677500
std        36.050009
min        10.000000
25%        38.000000
50%        54.000000
75%        78.000000
max       342.000000
Name: news, dtype: float64

In [36]:
train_raw.trigger.nunique()

1256

In [37]:
train_raw['trigger'].value_counts()>1

召开       True
发布       True
参加       True
接受       True
发表       True
        ...  
突发      False
稿酬      False
提前      False
赶回      False
没有遇到    False
Name: trigger, Length: 1256, dtype: bool

In [38]:
lb = (train_raw['trigger'].value_counts()>1).reset_index()
lb.columns = ['lb','flag']
lb = list(lb[lb['flag']==True].lb.values)
predicate2id = {j:i for i,j in enumerate(lb)}
id2predicate = {i:j for i,j in enumerate(lb)}

In [39]:
len(lb)

478

In [29]:
lb[:5]

['召开', '发布', '参加', '接受', '发表']

In [40]:
predicate2id

{'召开': 0,
 '发布': 1,
 '参加': 2,
 '接受': 3,
 '发表': 4,
 '举行': 5,
 '开展': 6,
 '提出': 7,
 '成立': 8,
 '推出': 9,
 '作出': 10,
 '发生': 11,
 '公布': 12,
 '到达': 13,
 '离开': 14,
 '建立': 15,
 '发现': 16,
 '组织': 17,
 '参与': 18,
 '恢复': 19,
 '启动': 20,
 '支持': 21,
 '找到': 22,
 '印发': 23,
 '出任': 24,
 '介绍': 25,
 '出席': 26,
 '制定': 27,
 '提供': 28,
 '增加': 29,
 '转入': 30,
 '设立': 31,
 '通报': 32,
 '宣布': 33,
 '举办': 34,
 '反对': 35,
 '指出': 36,
 '导致': 37,
 '形成': 38,
 '选择': 39,
 '批准': 40,
 '做出': 41,
 '购买': 42,
 '推动': 43,
 '推进': 44,
 '执行': 45,
 '调查': 46,
 '申请': 47,
 '关注': 48,
 '加入': 49,
 '分享': 50,
 '投入': 51,
 '给予': 52,
 '减少': 53,
 '解决': 54,
 '控制': 55,
 '制造': 56,
 '被确诊': 57,
 '检查': 58,
 '披露': 59,
 '打造': 60,
 '开启': 61,
 '会见': 62,
 '报道': 63,
 '改变': 64,
 '分析': 65,
 '进行': 66,
 '创造': 67,
 '调整': 68,
 '通过': 69,
 '带来': 70,
 '处理': 71,
 '影响': 72,
 '担任': 73,
 '建议': 74,
 '开发': 75,
 '告诉': 76,
 '提高': 77,
 '学习': 78,
 '发出': 79,
 '到': 80,
 '开放': 81,
 '前往': 82,
 '完成': 83,
 '抵达': 84,
 '建设': 85,
 '乘': 86,
 '实施': 87,
 '建成': 88,
 '发行': 89,
 '来到': 90,
 '解除': 91,

In [30]:
train = train_raw
train = train[train['trigger'].isin(lb)]
train['label_1'] = list(map(lambda x,y,z:(str(x),str(y),str(z)),train['object'],train['trigger'],train['subject']))
train = train.groupby('news')['label_1'].apply(list).reset_index()
train = list(map(lambda x,y:{'text':x,'spo_list':y},train['news'],train['label_1']))

train_data = train[:5000]
valid_data = train[5000:]

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
!cat ../../../Downloads/NER-de-dev.tsv | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > dev.txt.tmp

In [22]:
!cat ../../../Downloads/NER-de-train.tsv | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > train.txt.tmp

In [23]:
!cat ../../../Downloads/NER-de-test.tsv | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > test.txt.tmp

In [24]:
!cat ../../../Downloads/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [16]:
!python3 scripts/preprocess.py dev.txt.tmp $BERT_MODEL $MAX_LENGTH > dev.txt

python3: can't open file 'scripts/preprocess.py': [Errno 2] No such file or directory


In [19]:
import sys

from transformers import AutoTokenizer


dataset = 'dev.txt.tmp'
model_name_or_path = 'bert-base-multilingual-cased'
max_len = 128

subword_len_counter = 0

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
max_len -= tokenizer.num_special_tokens_to_add()
lines = []
with open(dataset, "rt") as f_p:
    for line in f_p:
        line = line.rstrip()

        if not line:
            print(line)
            subword_len_counter = 0
            continue

        token = line.split()[0]

        current_subwords_len = len(tokenizer.tokenize(token))

        # Token contains strange control characters like \x96 or \x95
        # Just filter out the complete line
        if current_subwords_len == 0:
            continue

        if (subword_len_counter + current_subwords_len) > max_len:
            print("")
            print(line)
            subword_len_counter = current_subwords_len
            continue

        subword_len_counter += current_subwords_len

#         print(line)
        lines.append(line)

I0803 16:24:24.828222 140735683666816 file_utils.py:39] PyTorch version 1.5.1 available.
I0803 16:24:28.761773 140735683666816 file_utils.py:55] TensorFlow version 2.2.0 available.


ImportError: dlopen(/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so, 2): Symbol not found: ____chkstk_darwin
  Referenced from: /Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so (which was built for Mac OS X 10.15)
  Expected in: /usr/lib/libSystem.B.dylib
 in /Users/luoyonggui/anaconda3/lib/python3.7/site-packages/tokenizers/tokenizers.cpython-37m-darwin.so